Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 2a TEST Weighted Average Ensembling

In [13]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os

In [14]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/test.tsv', sep='\t')
models = ['bert-large-uncased', 'roberta-large']
n_models = len(models)

epoch_string = '../15_epochs_large_model/saved_test_result_2b_final'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

# Loading up all of the results
best_f1 = pd.read_csv('../15_epochs_large_model/eval_testing/validation_stats/all_best_overall_f1_score.tsv', sep='\t')
best_f1



,Unnamed: 0,bert-large-uncased,roberta-large
0,0,0.788181,0.805198
1,1,0.798593,0.789957
2,2,0.771915,0.777964
3,3,0.813640,0.796570
4,4,0.784882,0.782647


In [15]:
# Retrieving all the predictions from the 
list_of_df = []
list_of_f1_score = []
sum_of_all_f1_score = 0

for model in models: 
    specific_model_row = []
    specific_model_row_f1 = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_test_result.tsv'
        
        specific_f1_score = best_f1.at[rnd, model]
        specific_model_row_f1.append(specific_f1_score)
        sum_of_all_f1_score += specific_f1_score
        
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)
    
    list_of_df.append(specific_model_row)
    list_of_f1_score.append(specific_model_row_f1)


In [16]:
# TAKING THE WEIGHTED AVERAGE OF DATA

weighted_avg_original_tweet_id_list = []
weighted_avg_original_sentence_list = []
weighted_avg_original_claim_list = []

weighted_avg_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['id']
    original_sentence = row['text']
    original_claim = row['claim']
    
    # transferring the labels over to final list
    weighted_avg_original_tweet_id_list.append(original_tweet_id)
    weighted_avg_original_sentence_list.append(original_sentence)
    weighted_avg_original_claim_list.append(original_claim)
    
    specific_row_value = 0.0
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):

            particular_row_df = list_of_df[model_num][rnd_num]
            row = particular_row_df.loc[(particular_row_df['id'] == original_tweet_id)]

            prediction = row['Premise'].values[0] * list_of_f1_score[model_num][rnd_num]
            specific_row_value += prediction

    specific_row_value = specific_row_value / sum_of_all_f1_score
    specific_row_result = int(round(specific_row_value))
    weighted_avg_predicted_results.append(specific_row_result)
    

In [17]:
# Saving it as a dataframe
formatted_weighted_avg_prediction_data = pd.DataFrame(zip(weighted_avg_original_tweet_id_list, weighted_avg_original_sentence_list, weighted_avg_original_claim_list, weighted_avg_predicted_results), columns=['id', 'text', 'Claim', 'Premise'])
formatted_weighted_avg_prediction_data

,id,text,Claim,Premise
0,1307558525371965442,@narendramodi @rajnathsingh Student ka bhi soa...,school closures,0
1,1247739239879467009,—échale un vistazo a esto… … a fair piece on...,stay at home orders,0
2,1242046510155653125,Why do think skilling women and girls is impor...,stay at home orders,0
3,1358446499949084675,To reduce the risk of the virus spreading as e...,school closures,1
4,1249740062775902208,I speak for a great many people when i say WE ...,stay at home orders,0
...,...,...,...,...
9950,1242516037628813314,StayAtHomeSaveLives 21daysLockdown StayAtHome ...,stay at home orders,0
9951,1242746919933415424,If this is true this is heartbreaking StayAtHo...,stay at home orders,0
9952,1276638598813679617,"855 Sunset Cove Dr, Winter Haven, FL 33880 3 B...",stay at home orders,0
9953,1243504288661270528,StayAtHomeSaveLives StayHomeStaySafe StayHome ...,stay at home orders,0


In [18]:
# Saving it as a tsv file
os.makedirs('../15_epochs_large_model/ultra_final_test_result', exist_ok=True)
formatted_weighted_avg_prediction_data.to_csv('../15_epochs_large_model/ultra_final_test_result/formatted_majority_data.tsv', sep='\t', index=False)